In [34]:
from collections import defaultdict

import operator
import os
import json
import re
import pandas as pd
from pandas import DataFrame

In [35]:
#Setting the data, output and challenge dataset path
Data_Path = '/Users/bijayani/Documents/MS/SEM3/CMPE256/Project/spotify_million_playlist_dataset/data'
Result_Path = '/Users/bijayani/Documents/MS/SEM3/CMPE256/Project/SmallerDataSet/TitleMatching/'
Challenge_data = '/Users/bijayani/Documents/MS/SEM3/CMPE256/Project/spotify_million_playlist_dataset_challenge/'

In [36]:
#Open a csv file in write mode.
final_file = open(os.path.join(Result_Path, 'title_popularity_recs.csv'), 'w')

In [37]:
#Normalized method is used to normal the titles of the playlist.
def normalize_title(title):
    title = title.lower()
    title = re.sub(r"[.,\/#!$%\^\*;:{}=\_`~()@]", ' ', title)
    title = re.sub(r'\s+', ' ', title).strip()
    return title

In [38]:
# Loading the challenge dataset!

#Creating a set for storing the challenge dataset playlist title.
cold_start_titles = set()

#load the challenge dataset
f = open(os.path.join(Challenge_data, "challenge_set.json"))
js = f.read()
challenge_set = json.loads(js)
f.close()

In [39]:
#check for the playlist that has 0 seed tracks. Those playlist will be cold start titles.
for playlist in challenge_set['playlists']:
    if playlist['num_samples'] < 1:
        title = normalize_title(playlist['name'])
        cold_start_titles.add(title)
        

In [40]:
#Read the mdp data slices
filenames = os.listdir(Data_Path)

In [41]:
#creating dictionary to save the titles
title_dict = dict()
final_dict = defaultdict(list)

In [42]:
#Create pid_playlistName list to save the pid and titles of mdp datset.
Pid_playListName =  []
for filename in sorted(filenames):
        #Fetching the mdp data.
        if filename.startswith("mpd.slice.") and filename.endswith(".json"):
            fullpath = os.sep.join((Data_Path, filename))
            f = open(fullpath)
            js = f.read()
            f.close()
            slice = json.loads(js)
            
            #Fetching each playlist from the mdp data file.
            for playlist in slice['playlists']:
                #Extracing the playlistid and playlist title
                pid = playlist['pid']
                title = normalize_title(playlist['name'])
                
                #Adding the playlist and playlist name i the Pid_playListName
                Pid_playListName.append([pid, title])
                
                #Check if this playlist name exist in the cold_start_titles, if the name existing then extracting tracks from this 
                # mpd playlist to use these tracks for the cold start titles.
                if title in cold_start_titles:
                    for i, track in enumerate(playlist['tracks']):
                        trackuri = track['track_uri']
                        if title not in title_dict.keys():
                            temp_dict = dict()
                            temp_dict[trackuri] = 1
                            title_dict[title] = temp_dict
                        else:
                            try:
                                title_dict[title][trackuri] += 1
                            except:
                                title_dict[title][trackuri] = 1

In [43]:
#Saving the pid and column name combination in the csv for analysis.
Pid_nameDF = DataFrame(Pid_playListName,columns=['pid', 'name'])
Pid_nameDF.to_csv('/Users/bijayani/Documents/MS/SEM3/CMPE256/Project/SmallerDataSet/TitleMatching/Pid_NameList')

In [44]:
#Append final_dict dictionary with the cold start titles and their respective tracks from the title_dict dictionary.
for title, items in title_dict.items():
        temp = []
        for track in items.keys():
            temp.append((track, items[track]))
        temp.sort(key=operator.itemgetter(1), reverse=True)
        temp1 = temp[:500]

        mapped_recs = [val[0] for val in temp1]
        #Notify if the mapped tracks for a title are less than 500 songs.
        if len(mapped_recs) < 500:
            print("Not enough items to recommend!")
            print(len(temp1), len(mapped_recs))
        final_dict[title] = mapped_recs

Not enough items to recommend!
384 384
Not enough items to recommend!
423 423
Not enough items to recommend!
436 436
Not enough items to recommend!
251 251
Not enough items to recommend!
427 427
Not enough items to recommend!
127 127
Not enough items to recommend!
406 406
Not enough items to recommend!
313 313
Not enough items to recommend!
457 457
Not enough items to recommend!
441 441
Not enough items to recommend!
242 242
Not enough items to recommend!
95 95
Not enough items to recommend!
286 286
Not enough items to recommend!
402 402
Not enough items to recommend!
236 236
Not enough items to recommend!
55 55
Not enough items to recommend!
20 20
Not enough items to recommend!
450 450
Not enough items to recommend!
249 249
Not enough items to recommend!
70 70
Not enough items to recommend!
126 126
Not enough items to recommend!
106 106
Not enough items to recommend!
468 468
Not enough items to recommend!
400 400


In [45]:
# write the the cold start title recommendations into a csv file for using it for other base models.
for title, recs in final_recs.items():
        str_out = title
        for rec in recs:
            str_out += ',' + str(rec)
        final_file.write(str_out + '\n')
        final_file.flush()
final_file.close()

In [48]:
#Fetching the title similarity csv to check for the 1000 challenge dataset playlist 
title  = pd.read_csv('/Users/bijayani/Documents/MS/SEM3/CMPE256/Project/SmallerDataSet/TitleMatching/title_popularity_recs.csv')

In [49]:
title.shape

(746, 501)